In [18]:
import datetime
import pandas as pd

##################################################################
# function
##################################################################


def minusDays(date, n_days):
    """Function that shifts the provided date backwards by n days."""
    if not isinstance(date, datetime.date):
        date = datetime.datetime.strptime(date, '%Y-%m-%d').date()
    return date - pd.Timedelta(days=n_days)


def getIncidentValues(df, companyID, fromDate, toDate):
    """Function that filters the incidents by company and date."""
    if not isinstance(fromDate, datetime.date):
        fromDate = datetime.datetime.strptime(fromDate, '%Y-%m-%d').date()
    if not isinstance(toDate, datetime.date):
        toDate = datetime.datetime.strptime(toDate, '%Y-%m-%d').date()

    incidents = df.loc[(df.company_id == companyID) &
                       (df.publication_date >= str(fromDate)) &
                       (df.publication_date <= str(toDate)),
                       ["publication_date", "severity", "novelty", "reach"]].copy()
    return incidents


def getLastIncidentDate(df, companyID, calculationStartDate):
    """Function that determines the date of the most recent incident linked to a company."""
    incidents = df[(df.company_id == companyID) &
                   (df.publication_date <= calculationStartDate)].copy()
    if incidents.empty:
        return None
    else:
        incidents.sort_values(by='publication_date',
                              ascending=False, inplace=True)
        return incidents.publication_date.iloc[0]


def getIncidentIntensity(date, incidentValues):
    """ Function that calculates the Incident Intensity.

    Parameters
    ----------
    date: pandas.Timestamp
    incidentValues: pandas.DataFrame

    Returns
    -------
    float
    """
    # count the number of incidents within the last 60 days
    incidentCount = incidentValues[(incidentValues.publication_date >= str(minusDays(date, 60)))
                                   & (incidentValues.publication_date <= str(date))].shape[0]
    # original
    # if incidentCount <= 8:
    #     return 0.5 + 0.1*incidentCount
    # elif incidentCount <= 12:
    #     return 1.3 + 0.05*(incidentCount - 8)
    # elif incidentCount <= 20:
    #     return 1.5 + 0.02*(incidentCount - 12)
    # else:
    #     return 1.66

    # for TEJ 
    if incidentCount <= 2:
        return 0.5 + 0.2*incidentCount
    elif incidentCount <= 5:    
        return 0.9 + 0.1*(incidentCount - 2)   
    elif incidentCount <= 10:    
        return 1.2 + 0.05*(incidentCount - 5) 
    elif incidentCount <= 20:    
        return 1.45 + 0.02*(incidentCount - 10)     
    else:
        return 1.65


def getTimeWeight(incidentDate, toDate):
    """Function that calculates the time-weight based on the incident date and the current calculation date."""
    days = int((pd.to_datetime(toDate) - pd.to_datetime(incidentDate)).days)
    return 1 - (days/PEAK_RRI_INTERVAL_DAYS)/2


def getAverageIncidentValue(date, incidentValues):
    """ Function that calculates the time-weighted Average Incident Value based on the incidents' severity, reach, and novelty.

    Parameters
    ----------
    date: pandas.Timestamp
    incidentValues: pandas.DataFrame

    Returns
    -------
    float
    """

    incidents = incidentValues[(incidentValues.publication_date >= str(minusDays(date, PEAK_RRI_INTERVAL_DAYS)))
                               & (incidentValues.publication_date <= str(date))].copy()
    if incidents.empty:
        return 0
    else:
        # calculate each incident's time weight
        incidents["time_weight"] = incidents.publication_date.apply(
            getTimeWeight, toDate=date)

        # calculate each incident's value (original)
        # incidents["incident_value"] = (
        #     (incidents.severity*incidents.reach*incidents.novelty)**(1/3))/3

        #  calc for TEJ 
        incidents["incident_value"] = ((incidents.severity*incidents.reach*incidents.novelty)**(0.5))/2

        # return the average incident value
        return (incidents["incident_value"]*incidents["time_weight"]).mean()


def getGeneralRRI(date, incidentValues):
    """Function that calculates the general RRI (Average Incident Value x Incident Intensity) of a company 
    for a specific date and given incidents."""

    incident_intensity = getIncidentIntensity(date, incidentValues)
    avg_incident_value = getAverageIncidentValue(date, incidentValues)
    generalRRI = 100*incident_intensity*avg_incident_value
    generalRRI = max(0, generalRRI)
    generalRRI = min(100, generalRRI)
    return generalRRI, incident_intensity, avg_incident_value
    

def getDecayedRRI(lastSignificantRRI, daysSinceLastIncident):
    """Function that applies the decay function to the general RRI."""
    # static variables
    DECAY_PLATEAU_INTERVAL_DAYS = 14
    RAPID_DECAY_RRI_THRESHOLD = 25
    RAPID_DECAY_RATE_DAYS = 60
    RAPID_DECAY_RATE = -RAPID_DECAY_RRI_THRESHOLD / RAPID_DECAY_RATE_DAYS  # = -0.42
    SLOW_DECAY_RATE_DAYS = 548
    SLOW_DECAY_RATE = -RAPID_DECAY_RRI_THRESHOLD / SLOW_DECAY_RATE_DAYS  # = -0.046

    def getNumberOfDaysToThresholdRRI(rri):
        """Function that calculates the days until the RRI threshold of 25 is reached."""
        return DECAY_PLATEAU_INTERVAL_DAYS + (RAPID_DECAY_RRI_THRESHOLD - rri)/RAPID_DECAY_RATE

    if lastSignificantRRI == 0:
        return lastSignificantRRI

    # value does not decay during plateau period
    if daysSinceLastIncident < DECAY_PLATEAU_INTERVAL_DAYS:
        return lastSignificantRRI
    # below here: daysSinceLastIncident always >= 14

    # calculate the number of days to go from the last incident to RRI = 25
    numberOfDaysToThresholdRRI = getNumberOfDaysToThresholdRRI(lastSignificantRRI)

    if daysSinceLastIncident < numberOfDaysToThresholdRRI:
        decayedRRI = (daysSinceLastIncident - DECAY_PLATEAU_INTERVAL_DAYS) * \
            RAPID_DECAY_RATE + lastSignificantRRI
    else:
        if lastSignificantRRI <= RAPID_DECAY_RRI_THRESHOLD:  # below or equal 25
            decayedRRI = (daysSinceLastIncident - DECAY_PLATEAU_INTERVAL_DAYS) * \
                SLOW_DECAY_RATE + lastSignificantRRI
        else:
            decayedRRI = (daysSinceLastIncident - numberOfDaysToThresholdRRI) * \
                SLOW_DECAY_RATE + RAPID_DECAY_RRI_THRESHOLD
    return max(decayedRRI, 0)


def calculateRRIhistory(companyID, toDate, incidentValueResult,
                        calculationStartDate, lastSignificantRRIDate):
    """Function that calculates the RRI history up to the current calculation date.

    Parameters
    ----------
    companyID : int
    toDate : pandas.Timestamp
    incidentValueResult: pandas.DataFrame
    calculationStartDate: pandas.Timestamp
    lastSignificantRRIDate: pandas.Timestamp

    Returns
    -------
    pandas.Series
    """
    if not isinstance(toDate, datetime.date):
        toDate = datetime.datetime.strptime(toDate, '%Y-%m-%d').date()

    # if there are no incidents, return an empty series
    if incidentValueResult.empty:
        return pd.Series()

    incidentValues = incidentValueResult.copy()

    # calculate the RRI for the last significant date
    lastSignificantRRI, incident_intensity, avg_incident_value = getGeneralRRI(lastSignificantRRIDate, incidentValues)

    # calculate the RRI for the calculation start date
    currentRRI = getDecayedRRI(lastSignificantRRI,
                               int((pd.to_datetime(calculationStartDate) - pd.to_datetime(lastSignificantRRIDate)).days))

    # loop through the calculation time range
    rriHistory = dict()
    rriList = list()
    dateList = list()
    incidentValuesList = list()
    timeweightList = list()
    intensityList = list()
    avg_incidentList = list()

    calculationDates = pd.date_range(calculationStartDate, toDate, freq="D")
    for date in calculationDates:
        date = pd.to_datetime(date).date()

        # check if there are incidents for the current date
        if (incidentValues.publication_date == str(date)).any():
            # if yes, calculate the general RRI (Average Incident Value x Incident Intensity)
            newGeneralRRI, incident_intensity, avg_incident_value = getGeneralRRI(date, incidentValues)

            # significance check
            if newGeneralRRI > currentRRI:
                lastSignificantRRI = newGeneralRRI
                lastSignificantRRIDate = date

        # calculate Current RRI (apply decay)
        currentRRI = getDecayedRRI(lastSignificantRRI,
                                   int((pd.to_datetime(date) - pd.to_datetime(lastSignificantRRIDate)).days))
        # rriHistory[date] = currentRRI
        
        rriList.append(currentRRI)
        dateList.append(date)
        # incidentValuesList.append(last_incident_value)
        # timeweightList.append(last_time_weight)
        intensityList.append(incident_intensity)
        avg_incidentList.append(avg_incident_value)

    # rriHistory = pd.Series(rriHistory)
    rriHistory['date'] = dateList
    rriHistory['RRI'] = rriList
    rriHistory['intensity'] = intensityList
    # rriHistory['incidentValues'] = incidentValuesList
    # rriHistory['timeweight'] = timeweightList
    rriHistory['avg_incident_value'] = avg_incidentList

    rriHistory = pd.DataFrame.from_dict(rriHistory)
    return rriHistory
    # return last_incident_value, last_time_weight
##################################################################
# main
##################################################################


def calculateCurrentRRI(df, companyID, date, PEAK_RRI_INTERVAL_DAYS):
    """Function that calculates the Current RRI of a company for the last n days.

    Parameters
    ----------
    companyID : int
        RepRisk Company ID
    date : pandas.Timestamp
        End date of the calculation window
    window: int
        Window size in days (default = 730)

    Returns
    -------
    float
    """
    cols = ["company_id", "publication_date", "severity", "novelty", "reach"]
    df = df[cols].copy()
    df["company_id"] = df["company_id"].apply(lambda x: str(x))
    df = df[df['company_id'] == str(companyID)].copy()

    calculationStartDate = df['publication_date'].iloc[0]

    # get the last incident from before the calculation start date
    lastSignificantRRIDate = getLastIncidentDate(
        df, companyID, calculationStartDate)
    if pd.isna(lastSignificantRRIDate):
        lastSignificantRRIDate = calculationStartDate

    # fetch all incidents up to 730 days before that date
    incidentValueResult = getIncidentValues(df, companyID,
                                            minusDays(
                                                lastSignificantRRIDate, PEAK_RRI_INTERVAL_DAYS),
                                            date)
    rriResults = calculateRRIhistory(companyID, date, incidentValueResult,
                                     calculationStartDate, lastSignificantRRIDate)
    # return rriResults
    return rriResults


# Data

In [21]:
from config import severityDict
data = pd.read_csv("data/TESGwithout0.csv")

data['company_id'] = data['company_id'].apply(lambda x: str(x))
data['severity'] = data['severity'].apply(lambda x: severityDict[x])
data['publication_date'] = pd.to_datetime(data['publication_date'])
data['publication_date'] = pd.to_datetime(data['publication_date'])
data

,company_id,company_name,SASB,publication_date,issue_main,severity,reach,novelty
0,1101,台泥,提煉與礦產加工,2018-01-31,E4 廢棄物及有毒物質管理,3,1,2
1,1101,台泥,提煉與礦產加工,2018-02-28,E4 廢棄物及有毒物質管理,1,1,2
2,1101,台泥,提煉與礦產加工,2018-03-31,E4 廢棄物及有毒物質管理,1,1,2
3,1101,台泥,提煉與礦產加工,2018-05-31,E4 廢棄物及有毒物質管理,1,1,2
4,1101,台泥,提煉與礦產加工,2018-06-30,E4 廢棄物及有毒物質管理,3,1,2
...,...,...,...,...,...,...,...,...
19635,9986,南天有線,服務,2020-06-17,G1 商業模式和創新,1,1,2
19636,9986,南天有線,服務,2020-08-13,G2 管理領導,3,1,2
19637,9986,南天有線,服務,2021-04-22,G2 管理領導,3,1,2
19638,9986,南天有線,服務,2021-08-12,G2 管理領導,3,1,2


# test case: 1101 台泥

In [36]:
compID = '1101'
fromDate = '2018-01-01'
toDate = '2023-01-31'
PEAK_RRI_INTERVAL_DAYS = 730

In [23]:
df_comp = calculateCurrentRRI(data, compID, toDate, PEAK_RRI_INTERVAL_DAYS)
df_comp.date = pd.to_datetime(df_comp.date)
df_comp['company_id'] = compID
df_comp['company_name'] = data[data['company_id'] == compID]['company_name'].iloc[0] 
df_comp['SASB'] = data[data['company_id'] == compID]['SASB'].iloc[0] 
df_comp['peak_2y'] = df_comp['RRI'].rolling(730, min_periods=1).max()
df_comp['trend_30d'] = df_comp['RRI'].diff(30)
df_comp = df_comp[['date', 'company_id', 'company_name', 'SASB', 'avg_incident_value', 'intensity', 'RRI', 'peak_2y', 'trend_30d']]
df_comp

,date,company_id,company_name,SASB,avg_incident_value,intensity,RRI,peak_2y,trend_30d
0,2018-01-31,1101,台泥,提煉與礦產加工,1.224745,0.7,85.732141,85.732141,NaN
1,2018-02-01,1101,台泥,提煉與礦產加工,1.224745,0.7,85.732141,85.732141,NaN
2,2018-02-02,1101,台泥,提煉與礦產加工,1.224745,0.7,85.732141,85.732141,NaN
3,2018-02-03,1101,台泥,提煉與礦產加工,1.224745,0.7,85.732141,85.732141,NaN
4,2018-02-04,1101,台泥,提煉與礦產加工,1.224745,0.7,85.732141,85.732141,NaN
...,...,...,...,...,...,...,...,...,...
1791,2022-12-27,1101,台泥,提煉與礦產加工,0.781325,1.0,28.910681,80.160681,-12.5
1792,2022-12-28,1101,台泥,提煉與礦產加工,0.781325,1.0,28.494014,80.160681,-12.5
1793,2022-12-29,1101,台泥,提煉與礦產加工,0.781325,1.0,28.077347,80.160681,-12.5
1794,2022-12-30,1101,台泥,提煉與礦產加工,0.781325,1.0,27.660681,80.160681,-12.5


# Calculating All companies

In [24]:
from tqdm import tqdm

In [47]:
fromDate = '2018-01-01'
toDate = '2023-01-31'
PEAK_RRI_INTERVAL_DAYS = 730

In [48]:
RRI_data = pd.DataFrame()
for compID in tqdm(data['company_id'].unique()):
    df_comp = calculateCurrentRRI(data, compID, toDate, PEAK_RRI_INTERVAL_DAYS)
    # 如果事件日超過設定日期範圍，會產生空的 dataframe，需要跳過計算下一個
    if df_comp.empty: 
        print(f"{compID} 事件日超過日期範圍")
        continue
    df_comp.date = pd.to_datetime(df_comp.date)
    df_comp['company_id'] = compID
    df_comp['company_name'] = data[data['company_id'] == compID]['company_name'].iloc[0] 
    df_comp['SASB'] = data[data['company_id'] == compID]['SASB'].iloc[0] 
    df_comp['peak_2y'] = df_comp['RRI'].rolling(730, min_periods=1).max()
    df_comp['trend_30d'] = df_comp['RRI'].diff(30)*(-1)
    df_comp = df_comp[['date', 'company_id', 'company_name', 'SASB', 'avg_incident_value', 'intensity', 'RRI', 'peak_2y', 'trend_30d']]
    RRI_data = pd.concat([RRI_data, df_comp])

100%|██████████| 2167/2167 [11:48<00:00,  3.06it/s]


In [43]:
RRI_data

,date,company_id,company_name,SASB,avg_incident_value,intensity,RRI,peak_2y,trend_30d
0,2018-05-31,6775,穎台科技,科技與通訊,0.707107,0.7,49.497475,49.497475,NaN
1,2018-06-01,6775,穎台科技,科技與通訊,0.707107,0.7,49.497475,49.497475,NaN
2,2018-06-02,6775,穎台科技,科技與通訊,0.707107,0.7,49.497475,49.497475,NaN
3,2018-06-03,6775,穎台科技,科技與通訊,0.707107,0.7,49.497475,49.497475,NaN
4,2018-06-04,6775,穎台科技,科技與通訊,0.707107,0.7,49.497475,49.497475,NaN
...,...,...,...,...,...,...,...,...,...
576,2022-01-27,6787,晶瑞光,科技與通訊,0.707107,0.7,2.043519,49.497475,1.368613
577,2022-01-28,6787,晶瑞光,科技與通訊,0.707107,0.7,1.997899,49.497475,1.368613
578,2022-01-29,6787,晶瑞光,科技與通訊,0.707107,0.7,1.952278,49.497475,1.368613
579,2022-01-30,6787,晶瑞光,科技與通訊,0.707107,0.7,1.906658,49.497475,1.368613


In [49]:
RRI_data.to_csv("output/rri.csv", encoding='utf_8_sig')

# Validation

In [51]:
pd.read_csv("output/rri.csv", parse_dates=True, index_col=0)

,date,company_id,company_name,SASB,avg_incident_value,intensity,RRI,peak_2y,trend_30d
0,2018-01-31,1101,台泥,提煉與礦產加工,1.224745,0.7,85.732141,85.732141,NaN
1,2018-02-01,1101,台泥,提煉與礦產加工,1.224745,0.7,85.732141,85.732141,NaN
2,2018-02-02,1101,台泥,提煉與礦產加工,1.224745,0.7,85.732141,85.732141,NaN
3,2018-02-03,1101,台泥,提煉與礦產加工,1.224745,0.7,85.732141,85.732141,NaN
4,2018-02-04,1101,台泥,提煉與礦產加工,1.224745,0.7,85.732141,85.732141,NaN
...,...,...,...,...,...,...,...,...,...
1744,2023-01-27,9986,南天有線,服務,0.748457,0.7,10.754598,59.850762,1.368613
1745,2023-01-28,9986,南天有線,服務,0.748457,0.7,10.708978,59.850762,1.368613
1746,2023-01-29,9986,南天有線,服務,0.748457,0.7,10.663357,59.850762,1.368613
1747,2023-01-30,9986,南天有線,服務,0.748457,0.7,10.617737,59.850762,1.368613


In [52]:
RRI_data[RRI_data['company_id']=='1101']

,date,company_id,company_name,SASB,avg_incident_value,intensity,RRI,peak_2y,trend_30d
0,2018-01-31,1101,台泥,提煉與礦產加工,1.224745,0.7,85.732141,85.732141,NaN
1,2018-02-01,1101,台泥,提煉與礦產加工,1.224745,0.7,85.732141,85.732141,NaN
2,2018-02-02,1101,台泥,提煉與礦產加工,1.224745,0.7,85.732141,85.732141,NaN
3,2018-02-03,1101,台泥,提煉與礦產加工,1.224745,0.7,85.732141,85.732141,NaN
4,2018-02-04,1101,台泥,提煉與礦產加工,1.224745,0.7,85.732141,85.732141,NaN
...,...,...,...,...,...,...,...,...,...
1822,2023-01-27,1101,台泥,提煉與礦產加工,0.781325,1.0,24.013943,80.160681,4.480071
1823,2023-01-28,1101,台泥,提煉與礦產加工,0.781325,1.0,23.968323,80.160681,4.109024
1824,2023-01-29,1101,台泥,提煉與礦產加工,0.781325,1.0,23.922702,80.160681,3.737978
1825,2023-01-30,1101,台泥,提煉與礦產加工,0.781325,1.0,23.877082,80.160681,3.366932
